In [2]:
# 数据处理
import pandas as pd
import numpy as np
import json
import csv
import random
import math
from itertools import product
from community import community_louvain



# 网络分析与可视化
import networkx as nx
from networkx.readwrite import json_graph
from pyvis import network as net
import matplotlib.pyplot as plt
import powerlaw # Power laws are probability distributions with the form:p(x)∝x−α

##  构建测试系统用的Master Concept Network

In [17]:
# with open('02_Graph/ConceptNetwork_Complete_CCFD-Similarity-Matrix.json') as f:
with open('02_Graph/ConceptNetwork_Complete_Word2Vec.json') as f:

    js_graph = json.load(f)
    G_ConceptNetwork = json_graph.node_link_graph(js_graph)

In [18]:
df_complete = pd.read_csv('01_Processed Data/Complete-Data.csv')

## 查找概念间共有的语义特征
def Extract_Associate_SemanticFeature(df, entity_col, association_col, concept_A, concept_B):
    listA = list(df[df[entity_col]==concept_A][association_col].unique())
    listB = list(df[df[entity_col]==concept_B][association_col].unique())
    return set(listA)&set(listB)

In [13]:
# # 测试Edge Filter的参数用。用完整的，未经过边筛选的 G_ConceptNetwork
# for u,v,e in G_ConceptNetwork.edges(data=True):
#     if ((e['weight'] < 0.63) & (e['weight']>0.62)):
#         shared_features = Extract_Associate_SemanticFeature(df_complete, 'Concept', 'Feature', u, v)
#         if shared_features !=0:
#             print(e['weight'],u,v,shared_features)

0.620441052 安全带 安全帽 {'有-扣子', '可以-保护人', '有-多种样式', '是-安保的', '是-工具', '可以-保护', '是-金属'}
0.622985581 安全帽 护目镜 {'有-带子', '可以-戴', '是-塑料', '需要-清洗', '有-多种种类', '有-多种颜色', '是-安保的', '是-用具', '可以-保护', '有-多种样式'}
0.624540571 鹌鹑 苍蝇 {'有-爪子', '可以-飞', '需要-空气', '有-嘴巴', '有-眼睛', '需要-食物', '需要-水', '是-动物', '有-翅膀'}
0.621430591 鹌鹑 鹅 {'有-爪子', '是-食物', '可以-飞', '需要-睡觉', '可以-下蛋', '需要-空气', '可以-游泳', '有-嘴巴', '有-眼睛', '像-鸡', '需要-食物', '需要-水', '有-羽毛', '是-动物', '有-翅膀', '有-毛', '是-鸟', '有-四肢'}
0.621996042 鹌鹑 火鸡 {'有-爪子', '是-食物', '可以-飞', '需要-睡觉', '需要-空气', '有-嘴巴', '有-眼睛', '需要-食物', '需要-虫', '需要-水', '有-羽毛', '是-动物', '有-翅膀', '有-毛', '是-鸟', '有-四肢'}
0.625812836 鹌鹑 鸭子 {'是-食物', '可以-飞', '需要-睡觉', '可以-下蛋', '需要-空气', '可以-游泳', '有-嘴巴', '有-眼睛', '像-鸡', '需要-食物', '需要-水', '有-羽毛', '是-动物', '是-卵生的', '有-翅膀', '有-毛', '可以-捕猎'}
0.623551032 鹌鹑 野鸡 {'有-爪子', '是-食物', '可以-飞', '需要-睡觉', '可以-下蛋', '需要-空气', '有-嘴巴', '有-眼睛', '需要-食物', '需要-虫', '需要-水', '有-羽毛', '是-动物', '是-卵生的', '有-翅膀', '是-鸟'}
0.622278767 按键 电路板 {'有-多种型号', '需要-设计', '需要-制造', '有-多种功能', '有-多种种类', '需要-电', '是-工具', '需要-安装'

In [19]:
def filter_edge(u,v):
    # if G_ConceptNetwork[u][v]['weight'] > 0.52:
    if G_ConceptNetwork[u][v]['weight'] > 0.62:

    # if G[u][v]['weight'] > 0:
        return G_ConceptNetwork[u][v]

G_Subgraph = nx.subgraph_view(G_ConceptNetwork,filter_edge=filter_edge,)
largest_cc = max(nx.connected_components(G_Subgraph), key=len)
G_MasterConceptNetwork = G_Subgraph.subgraph(largest_cc)


# # 将全图导出
# with open('02_Graph/ConceptNetwork_Complete_CCFD-Similarity-Matrix_Weight-0.52.json','w+') as f:
with open('02_Graph/ConceptNetwork_Complete_Word2Vec-0.62.json','w+') as f:
    f.write(json.dumps(nx.node_link_data(G_ConceptNetwork), ensure_ascii=False))


# 将全图导出 for Gephi
# nx.write_gexf(G_MasterConceptNetwork, "02_Graph/ConceptNetwork_Complete_CCFD-Similarity-Matrix_Weight-0.52.gexf")
nx.write_gexf(G_MasterConceptNetwork, "02_Graph/ConceptNetwork_Complete_Word2Vec-0.62.gexf")

In [20]:
len(G_MasterConceptNetwork.nodes)

1195

## Small-World Network Test

In [7]:
## To Do 很久

# nx.average_shortest_path_length(G_MasterConceptNetwork)

In [ ]:
# # Power law test

# degree_sequence = sorted([d for n, d in G_ConceptNetwork.degree()], reverse=True) # used for degree distribution and powerlaw test
# fit = powerlaw.Fit(degree_sequence) 
# fit.power_law.alpha
# fit.power_law.sigma
# fit.distribution_compare('power_law', 'exponential')
# # fig2 = fit.plot_pdf(color='b', linewidth=2)
# # fit.power_law.plot_pdf(color='g', linestyle='--', ax=fig2)


## 计算图指标 Graph Metrics

### degree / betweennes_centrality / closeness_centrality / laplacian_centrality / clustering coefficent / pagerank

In [21]:
# 两个值域区间映射
def translate(value, leftMin, leftMax, rightMin, rightMax):
    # Figure out how 'wide' each range is
    leftSpan = leftMax - leftMin
    rightSpan = rightMax - rightMin

    # Convert the left range into a 0-1 range (float)
    valueScaled = float(value - leftMin) / float(leftSpan)

    # Convert the 0-1 range into a value in the right range.
    return rightMin + (valueScaled * rightSpan)



In [22]:
# 计算 degree centrality 很快
degree_centrality = nx.degree_centrality(G_MasterConceptNetwork)

# 重映射值域
left_min = min(degree_centrality.values())
left_max = max(degree_centrality.values())
degree_centrality.update({x: translate(y,left_min,left_max, 0, 1.0)for x, y in degree_centrality.items()})


# 写入节点属性
nx.set_node_attributes(G_MasterConceptNetwork, degree_centrality, "degree_centrality")


In [23]:
# 计算 pagerank 很快
pagerank = nx.pagerank(G_MasterConceptNetwork, alpha=0.9, weight='weight')

# 重映射值域
left_min = min(pagerank.values())
left_max = max(pagerank.values())
pagerank.update({x: translate(y,left_min,left_max, 0, 1.0)for x, y in pagerank.items()})


# 写入节点属性
nx.set_node_attributes(G_MasterConceptNetwork, pagerank, "pagerank")

In [119]:
pagerank

{'安全带': 0.06370626456059618,
 '安全帽': 0.10192888088667294,
 '八宝粥': 0.15739519696410395,
 '巴士': 0.27908439714988525,
 '芭蕾舞裙': 0.2594435914170281,
 '白菜': 0.3087510653130814,
 '白饭': 0.16185794569280337,
 '白果': 0.1453061325312306,
 '白桦树': 0.1630306950387183,
 '白酒': 0.21937958271163074,
 '白萝卜': 0.319737972643026,
 '白皮松': 0.18641158530592014,
 '白砂糖': 0.2599427943657558,
 '白水': 0.12432425773034256,
 '白糖': 0.17250765177836605,
 '白杨': 0.18925120391317637,
 '白粥': 0.21032107263772692,
 '百合': 0.3103526295865533,
 '百事可乐': 0.2787300596485677,
 '百香果': 0.1435936120228154,
 '柏树': 0.20147665345612964,
 '柏油路': 0.19140586150187644,
 '板栗': 0.20568910461374576,
 '棒棒糖': 0.10554580356485406,
 '包': 0.0972975794795142,
 '包装纸': 0.08276902185335673,
 '包子': 0.23084629213627184,
 '薄荷': 0.2007334209411101,
 '宝石': 0.22012820750640072,
 '保险柜': 0.05201128113472782,
 '报警器': 0.30631502897109214,
 '爆米花': 0.17291020538434476,
 '杯垫': 0.3782095800401094,
 '杯子': 0.3635956966245285,
 '背包': 0.1220451964482083,
 '背带裤': 0.19299579

In [24]:
# 计算 closeness centrality 时间很长12min
closeness_centrality = nx.closeness_centrality(G_MasterConceptNetwork)

# 重映射值域
left_min = min(closeness_centrality.values())
left_max = max(closeness_centrality.values())
closeness_centrality.update({x: translate(y,left_min,left_max, 0, 1.0)for x, y in closeness_centrality.items()})

# 写入节点属性
nx.set_node_attributes(G_MasterConceptNetwork, closeness_centrality, "closeness_centrality")


In [135]:
# 计算 betweenness centrality 时间很长
betweenness_centrality = nx.betweenness_centrality(G_MasterConceptNetwork)

# 重映射值域
left_min = min(betweenness_centrality.values())
left_max = max(betweenness_centrality.values())
betweenness_centrality.update({x: translate(y,left_min,left_max, 0, 1.0) for x, y in betweenness_centrality.items()})

# 写入节点属性（先用0代替）
# nx.set_node_attributes(G_MasterConceptNetwork, 0, "betweenness_centrality")
# nx.set_node_attributes(G_MasterConceptNetwork, betweenness_centrality, "closeness_centrality")


In [25]:
# 计算 Laplacian centrality
# https://andrewpwheeler.com/2015/07/29/laplacian-centrality-in-networkx-python/

def lap_energy(graph, weight='weight'):
  # degrees = graph.degree(weight=weight)
  degrees = {node:val for (node, val) in graph.degree(weight=weight)}

  d1 = sum(v**2 for i,v in degrees.items())
  wl = 0
  for i in graph.edges(data=True):
    wl += (i[2].get(weight))**2
  return [d1,2*wl]

def cw(graph,node,weight='weight'):
  neis = graph.neighbors(node)
  ne = graph[node]
  cw,sub = 0,0
  for i in neis:
    we = ne[i].get(weight)
    od = graph.degree(i,weight=weight)
    sub += -od**2 + (od - we)**2
    cw += we**2
  return [cw,sub]

def lap_cent_weighted(graph, nbunch=None, norm=False, weight='weight'):
  if nbunch is None:
    vs = graph.nodes()
  else:
    vs = nbunch
  if norm == True:
    fe = lap_energy(graph,weight=weight)
    den = float(fe[0]+fe[1])
  else:
    den = 1
  result = {}
  for i in vs:
     d2 = graph.degree(i,weight=weight)
     w2 = cw(graph,i,weight=weight)
     fin = d2**2 - w2[1] + 2*w2[0]
     result[i]=fin/den
  return result

laplacian_centrality = lap_cent_weighted(G_MasterConceptNetwork, norm=True)

# 重映射值域
left_min = min(laplacian_centrality.values())
left_max = max(laplacian_centrality.values())
laplacian_centrality.update({x: translate(y,left_min,left_max, 0, 1.0)for x, y in laplacian_centrality.items()})


# 写入节点属性
nx.set_node_attributes(G_MasterConceptNetwork, laplacian_centrality, "laplacian_centrality")


In [26]:
# To Do
# 计算 clustering coefficient
clustering_coefficent = nx.clustering(G_MasterConceptNetwork, weight='weight')

# 重映射值域
left_min = min(clustering_coefficent.values())
left_max = max(clustering_coefficent.values())
clustering_coefficent.update({x: translate(y,left_min,left_max, 0, 1.0)for x, y in clustering_coefficent.items()})

# 写入节点属性
nx.set_node_attributes(G_MasterConceptNetwork, clustering_coefficent, "clustering_coefficent")
# nx.set_node_attributes(G_MasterConceptNetwork, betweenness_centrality, "closeness_centrality")

In [27]:
# 计算 louvain community
louvain_community = community_louvain.best_partition(G_MasterConceptNetwork)

# 写入节点属性
nx.set_node_attributes(G_MasterConceptNetwork, louvain_community, "cluster")

In [133]:
G_MasterConceptNetwork.nodes['拇指']

{'subcategory': '身体部位',
 'maincategory': '身体部位',
 'degree_centrality': 0.1927710843373494,
 'pagerank': 0.3236200709635144,
 'closeness_centrality': 0.6674584631732533,
 'laplacian_centrality': 0.05800476912071978,
 'clustering_coefficent': 0.4445809158938773,
 'cluster': 17,
 'betweenness_centrality': 0}

## 在Gephi中计算布局

In [29]:
nx.write_gexf(G_MasterConceptNetwork, "02_Graph/MasterConceptNetwork_Word2Vec-0.62.gexf")
# nx.write_gexf(G_MasterConceptNetwork, "02_Graph/MasterConceptNetwork.gexf")

------ Gephi Force Atalas 2 Layout / Export yo graphml ------

In [30]:
# From Gephi to G6 
# G=nx.read_graphml('02_Graph/MasterConceptNetwork.graphml')
G=nx.read_graphml('02_Graph/MasterConceptNetwork_Word2Vec-0.62.graphml')


for node in G:
    del G.nodes[node]['r']
    del G.nodes[node]['g']
    del G.nodes[node]['b']
    del G.nodes[node]['size']
    del G.nodes[node]['label']

nx.node_link_data(G)

{'directed': False,
 'multigraph': False,
 'graph': {'node_default': {}, 'edge_default': {}},
 'nodes': [{'subcategory': '工具',
   'maincategory': '人造物',
   'degree_centrality': 0.05494505494505494,
   'pagerank': 0.06060983413463747,
   'closeness_centrality': 0.5241060346953804,
   'laplacian_centrality': 0.021495841590250742,
   'clustering_coefficent': 0.23774567830802595,
   'cluster': 0,
   'x': 1936.9679,
   'y': -3296.95,
   'id': '安全带'},
  {'subcategory': '工具',
   'maincategory': '人造物',
   'degree_centrality': 0.06593406593406592,
   'pagerank': 0.07883389922106622,
   'closeness_centrality': 0.5336790650928026,
   'laplacian_centrality': 0.020412418138692785,
   'clustering_coefficent': 0.17097677496747904,
   'cluster': 0,
   'x': 1628.231,
   'y': -3678.653,
   'id': '安全帽'},
  {'subcategory': '鸟',
   'maincategory': '动物',
   'degree_centrality': 0.15384615384615383,
   'pagerank': 0.17357224468394705,
   'closeness_centrality': 0.6579884611292969,
   'laplacian_centrality': 

In [31]:
for node, data in G.nodes(data=True):
    data["forceAtalasX"] = data.pop("x")
    data["forceAtalasY"] = data.pop("y")

------ yEd Balloon Layout / Export yo graphml ------

In [24]:
G=nx.read_graphml('02_Graph/MasterConceptNetwork_Word2Vec-0.62.graphml')
G_Tree_Ballon=nx.read_graphml('02_Graph/tree_balloon.graphml')

In [25]:
for node, data in G.nodes(data=True):
    data["forceAtalasX"] = data.pop("x")
    data["forceAtalasY"] = data.pop("y")

In [26]:
for node, data in G.nodes(data=True):
    index = [n for n,d in G_Tree_Ballon.nodes(data=True) if d.get('label')==node][0]
    data["balloonX"] = G_Tree_Ballon.nodes[index]['x']
    data["balloonY"] = G_Tree_Ballon.nodes[index]['y']

In [27]:
G.nodes['安全带']

{'label': '安全带',
 'subcategory': '工具',
 'maincategory': '人造物',
 'degree_centrality': 0.05494505494505494,
 'pagerank': 0.06060983413463747,
 'closeness_centrality': 0.5241060346953804,
 'laplacian_centrality': 0.021495841590250742,
 'clustering_coefficent': 0.23774567830802595,
 'cluster': 0,
 'size': 14.945055,
 'r': 0,
 'g': 0,
 'b': 0,
 'forceAtalasX': 1936.9679,
 'forceAtalasY': -3296.95,
 'balloonX': '2879.8391723214136',
 'balloonY': '3239.5520541934743'}

In [28]:
# 导出 for G6
# with open('02_Graph/MasterConceptNetwork_G6.json','w+') as f:
with open('02_Graph/MasterConceptNetwork_Word2Vec-0.62_G6.json','w+') as f:
    f.write(json.dumps(nx.node_link_data(G), ensure_ascii=False).replace('links','edges'))

# 导出 for networkx
# with open('02_Graph/MasterConceptNetwork.json','w+') as f:
with open('02_Graph/MasterConceptNetwork_Word2Vec-0.62.json','w+') as f:
    f.write(json.dumps(nx.node_link_data(G), ensure_ascii=False))